In [32]:
import math
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats import pearsonr
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
warnings.filterwarnings("ignore")
# https://www.kaggle.com/code/fuzzywizard/rec-sys-collaborative-filtering-dl-techniques

In [2]:
%cd Z:\PPMI_Data\Excels\CollaborativeFiltering
df = pd.read_csv('Feats45_unCategAge_APPRDX.csv')
df.shape
df.head(2)

Z:\PPMI_Data\Excels\CollaborativeFiltering


,PATNO,Patient_ID,APPRDX,Age,Anxiety,Apathy,Benton,Clock,Cognition,COGSTATE,...,Semantic,SleepDay,SleepNight,STAIA,STAIS,Symbol_Digit,Trail_Making_A,Trail_Making_B,UPSIT,Urine
0,3002,1,1,67.578082,1,0,12.16,7,1,1,...,57,1,0,45,59,47.5,110,204,17,1
1,3010,2,1,46.965716,1,1,7.76,6,1,1,...,36,1,3,40,39,52.0,27,52,9,0


In [3]:
# Removing a column
df1 = df.drop(['PATNO','APPRDX'], axis = 1)
df1.head(2)

,Patient_ID,Age,Anxiety,Apathy,Benton,Clock,Cognition,COGSTATE,Constipate,Depress,...,Semantic,SleepDay,SleepNight,STAIA,STAIS,Symbol_Digit,Trail_Making_A,Trail_Making_B,UPSIT,Urine
0,1,67.578082,1,0,12.16,7,1,1,1,1,...,57,1,0,45,59,47.5,110,204,17,1
1,2,46.965716,1,1,7.76,6,1,1,0,0,...,36,1,3,40,39,52.0,27,52,9,0


In [4]:
df2 = pd.read_csv('NonMotor_Empty.csv')
df2.shape
df2.head(2)

,Patient_ID,Anxiety,Apathy,Benton,Clock,Cognition,COGSTATE,Constipate,Depress,DopaDefic,...,Semantic,SleepDay,SleepNight,STAIA,STAIS,Symbol_Digit,Trail_Making_A,Trail_Making_B,UPSIT,Urine
0,999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Removing a column
df3 = df2.drop('Patient_ID', axis = 1)
df3.head(2)

,Anxiety,Apathy,Benton,Clock,Cognition,COGSTATE,Constipate,Depress,DopaDefic,Education,...,Semantic,SleepDay,SleepNight,STAIA,STAIS,Symbol_Digit,Trail_Making_A,Trail_Making_B,UPSIT,Urine
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Ask the user for the updated value of age
new_age = int(input("What is the user's age (in years): "))

# Update the value of age in the dataframe
df3.at[0, 'Age'] = new_age

# Drop the 46th column from the original DataFrame
col_age = df3.pop('Age')

# Insert the column at the 5th position
df3.insert(0, "Age", col_age)
df3.head(2)

What is the user's age (in years): 48


,Age,Anxiety,Apathy,Benton,Clock,Cognition,COGSTATE,Constipate,Depress,DopaDefic,...,Semantic,SleepDay,SleepNight,STAIA,STAIS,Symbol_Digit,Trail_Making_A,Trail_Making_B,UPSIT,Urine
0,48.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
#Below will take at least 3 features

In [8]:
def update_values(df3):
    # Input from the user for number of columns
    num_columns = int(input("Enter the number of columns you want to update (minimum 3, maximum 45): "))
    
    while num_columns < 3 or num_columns > 45:
        print("Invalid number of columns. Please enter a number between 3 and 45.")
        num_columns = int(input("Enter the number of columns you want to update (minimum 3, maximum 45): "))
    
    columns = []
    values = []
     
     # Loop to get input from the user for each column
    for i in range(num_columns):
        column = input("Enter the name of the Feature %d: " % (i + 1))
        value = input("Enter the score of the Feature %d: " % (i + 1))
        
        columns.append(column)
        values.append(int(value))
    
    # Update the values in the dataframe
    for i in range(num_columns):
        df3.loc[df3[columns[i]] != values[i], columns[i]] = values[i]
    
    return df3

# Call the function
df3 = update_values(df3)

# Print the updated dataframe
df3.head()

Enter the number of columns you want to update (minimum 3, maximum 45): 3
Enter the name of the Feature 1: Clock
Enter the score of the Feature 1: 6
Enter the name of the Feature 2: UPSIT
Enter the score of the Feature 2: 35
Enter the name of the Feature 3: Pain
Enter the score of the Feature 3: 1


,Age,Anxiety,Apathy,Benton,Clock,Cognition,COGSTATE,Constipate,Depress,DopaDefic,...,Semantic,SleepDay,SleepNight,STAIA,STAIS,Symbol_Digit,Trail_Making_A,Trail_Making_B,UPSIT,Urine
0,48.0,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.0,NaN


In [9]:
# Adding back the column (as the first column)
df3.insert (0, 'Patient_ID', df2['Patient_ID'])
df3.head()

,Patient_ID,Age,Anxiety,Apathy,Benton,Clock,Cognition,COGSTATE,Constipate,Depress,...,Semantic,SleepDay,SleepNight,STAIA,STAIS,Symbol_Digit,Trail_Making_A,Trail_Making_B,UPSIT,Urine
0,999,48.0,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.0,NaN


In [10]:
# merging two dataframes [original and our input dataframes]
merged_df = pd.concat ([df1, df3], axis = 0)
merged_df.reset_index(inplace = True)
merged_df.tail()

,index,Patient_ID,Age,Anxiety,Apathy,Benton,Clock,Cognition,COGSTATE,Constipate,...,Semantic,SleepDay,SleepNight,STAIA,STAIS,Symbol_Digit,Trail_Making_A,Trail_Making_B,UPSIT,Urine
246,246,248,57.447728,0.0,0.0,14.77,6.0,0.0,1.0,3.0,...,47.0,0.0,0.0,46.0,44.0,56.250000,27.0,75.0,36.0,0.0
247,247,249,58.260371,0.0,0.0,14.18,6.0,0.0,1.0,0.0,...,58.0,1.0,3.0,47.0,40.0,58.750000,28.0,48.0,38.0,1.0
248,248,250,66.383562,0.0,0.0,11.06,7.0,0.0,1.0,0.0,...,49.0,2.0,1.0,45.0,46.0,45.830002,36.0,106.0,32.0,1.0
249,249,251,68.073973,0.0,0.0,11.06,7.0,0.0,1.0,0.0,...,57.0,0.0,3.0,44.0,45.0,50.000000,30.0,80.0,38.0,0.0
250,0,999,48.000000,NaN,NaN,NaN,6.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.0,NaN


In [11]:
# dropping a few columns
merged_df1 = merged_df.drop(['index','Patient_ID'], axis = 1)
merged_df1.head(2)


,Age,Anxiety,Apathy,Benton,Clock,Cognition,COGSTATE,Constipate,Depress,DopaDefic,...,Semantic,SleepDay,SleepNight,STAIA,STAIS,Symbol_Digit,Trail_Making_A,Trail_Making_B,UPSIT,Urine
0,67.578082,1.0,0.0,12.16,7.0,1.0,1.0,1.0,1.0,0.0,...,57.0,1.0,0.0,45.0,59.0,47.5,110.0,204.0,17.0,1.0
1,46.965716,1.0,1.0,7.76,6.0,1.0,1.0,0.0,0.0,0.0,...,36.0,1.0,3.0,40.0,39.0,52.0,27.0,52.0,9.0,0.0


In [12]:
# scaling values of every column from 1 to 5 [column wise]
scaler = MinMaxScaler (feature_range = (1,5))
df_scaled = scaler.fit_transform(merged_df1)
df_scaled = pd.DataFrame (df_scaled, columns = merged_df1.columns)
df_scaled.head(2)

,Age,Anxiety,Apathy,Benton,Clock,Cognition,COGSTATE,Constipate,Depress,DopaDefic,...,Semantic,SleepDay,SleepNight,STAIA,STAIS,Symbol_Digit,Trail_Making_A,Trail_Making_B,UPSIT,Urine
0,3.786116,2.333333,1.0,3.503546,5.000000,3.0,1.0,2.333333,2.0,1.0,...,3.285714,3.0,1.0,3.434783,5.0,2.833333,3.832117,3.686131,2.333333,2.0
1,2.232411,2.333333,3.0,1.943262,4.333333,3.0,1.0,1.000000,1.0,1.0,...,1.785714,3.0,4.0,2.565217,3.0,3.133333,1.408759,1.467153,1.363636,1.0


In [13]:
# adding the column [as the first column]
df_scaled.insert (0, 'Patient_ID', merged_df['Patient_ID'], True)
df_scaled.head(3)

,Patient_ID,Age,Anxiety,Apathy,Benton,Clock,Cognition,COGSTATE,Constipate,Depress,...,Semantic,SleepDay,SleepNight,STAIA,STAIS,Symbol_Digit,Trail_Making_A,Trail_Making_B,UPSIT,Urine
0,1,3.786116,2.333333,1.0,3.503546,5.000000,3.0,1.0,2.333333,2.0,...,3.285714,3.0,1.0,3.434783,5.0,2.833333,3.832117,3.686131,2.333333,2.0
1,2,2.232411,2.333333,3.0,1.943262,4.333333,3.0,1.0,1.000000,1.0,...,1.785714,3.0,4.0,2.565217,3.0,3.133333,1.408759,1.467153,1.363636,1.0
2,3,4.267435,1.000000,1.0,3.975177,5.000000,3.0,5.0,1.000000,1.0,...,2.428571,3.0,3.0,2.913043,3.3,2.555333,3.248175,4.386861,3.424242,1.0


In [14]:
df_scaled.tail(3)

,Patient_ID,Age,Anxiety,Apathy,Benton,Clock,Cognition,COGSTATE,Constipate,Depress,...,Semantic,SleepDay,SleepNight,STAIA,STAIS,Symbol_Digit,Trail_Making_A,Trail_Making_B,UPSIT,Urine
248,250,3.696076,1.0,1.0,3.113475,5.000000,1.0,1.0,1.0,1.0,...,2.714286,5.0,2.0,3.434783,3.7,2.722,1.671533,2.255474,4.151515,2.0
249,251,3.823494,1.0,1.0,3.113475,5.000000,1.0,1.0,1.0,2.0,...,3.285714,1.0,4.0,3.260870,3.6,3.000,1.496350,1.875912,4.878788,1.0
250,999,2.310372,NaN,NaN,NaN,4.333333,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.515152,NaN


In [15]:
# using melt function [reshape]
df_melted = df_scaled.melt(id_vars='Patient_ID', var_name='Feature', value_name='Value')
print(df_melted.shape)
df_melted.head(2)

(11546, 3)


,Patient_ID,Feature,Value
0,1,Age,3.786116
1,2,Age,2.232411


In [16]:
#1. take groupby with respect to Feature and get mean avg of each Feature. 
feat_rating_count = df_melted.groupby('Patient_ID')['Value'].count()

#2. convert into dataframe for better operation work.
feat_rating_count = pd.DataFrame(feat_rating_count)

#3. change column name.
feat_rating_count.columns = ['Value_count']

#4. create new column Patient_ID
feat_rating_count['Patient_ID'] = feat_rating_count.index
feat_rating_count = feat_rating_count.reset_index(drop=True)

In [17]:
#merging to Rating dataframe so we have collectively all information together..
dframe = df_melted.merge(feat_rating_count, on = 'Patient_ID', how = 'inner')
print(dframe.shape)
dframe.head(2)

(11546, 4)


,Patient_ID,Feature,Value,Value_count
0,1,Age,3.786116,46
1,1,Anxiety,2.333333,46


In [18]:
#create a pivot matrix for user and Feature based on rating value..
RatingMat = dframe.pivot_table(index=['Patient_ID'],columns=['Feature'],values=['Value'],fill_value=0)

#keep a copy of original matrix for future refrence..
Original_RatingMat = RatingMat.copy()
RatingMat.head()

Value                                                          \
Feature          Age   Anxiety Apathy    Benton COGSTATE     Clock Cognition   
Patient_ID                                                                     
1           3.786116  2.333333      1  3.503546        1  5.000000         3   
2           2.232411  2.333333      3  1.943262        1  4.333333         3   
3           4.267435  1.000000      1  3.975177        5  5.000000         3   
4           3.521970  1.000000      1  2.659574        5  4.333333         3   
5           4.052632  2.333333      1  3.503546        1  5.000000         3   

                                         ...                            \
Feature    Constipate Depress DopaDefic  ...     STAIA STAIS  Semantic   
Patient_ID                               ...                             
1            2.333333       2         1  ...  3.434783   5.0  3.285714   
2            1.000000       1         1  ...  2.565217   3.0  1.785714   
3            1.000000       1         1  ...  2.913043   3.3  2.428571   
4            1.000000       1         1  ...  4.130435   3.8  2.714286   
5            1.000000       2         1  ...  3.956522   3.7  3.357143   

                                                                           \
Feature    SleepDay SleepNight Symbol_Digit Trail_Making_A Trail_Making_B   
Patient_ID                                                                  
1                 3          1     2.833333       3.832117       3.686131   
2                 3          4     3.133333       1.408759       1.467153   
3                 3          3     2.555333       3.248175       4.386861   
4                 3          3     2.500000       2.576642       3.116788   
5                 5          1     3.200000       1.817518       2.547445   

                            
Feature        UPSIT Urine  
Patient_ID                  
1           2.333333     2  
2           1.363636     1  
3           3.424242     1  
4           1.242424     2  
5           3.181818     1  

[5 rows x 46 columns]

In [19]:
#drop multilevel column for better usabiltity...
RatingMat.columns = RatingMat.columns.droplevel()
RatingMat.head(5)

Feature,Age,Anxiety,Apathy,Benton,COGSTATE,Clock,Cognition,Constipate,Depress,DopaDefic,...,STAIA,STAIS,Semantic,SleepDay,SleepNight,Symbol_Digit,Trail_Making_A,Trail_Making_B,UPSIT,Urine
Patient_ID,,,,,,,,,,,,,,,,,,,,,
1,3.786116,2.333333,1,3.503546,1,5.000000,3,2.333333,2,1,...,3.434783,5.0,3.285714,3,1,2.833333,3.832117,3.686131,2.333333,2
2,2.232411,2.333333,3,1.943262,1,4.333333,3,1.000000,1,1,...,2.565217,3.0,1.785714,3,4,3.133333,1.408759,1.467153,1.363636,1
3,4.267435,1.000000,1,3.975177,5,5.000000,3,1.000000,1,1,...,2.913043,3.3,2.428571,3,3,2.555333,3.248175,4.386861,3.424242,1
4,3.521970,1.000000,1,2.659574,5,4.333333,3,1.000000,1,1,...,4.130435,3.8,2.714286,3,3,2.500000,2.576642,3.116788,1.242424,2
5,4.052632,2.333333,1,3.503546,1,5.000000,3,1.000000,2,1,...,3.956522,3.7,3.357143,5,1,3.200000,1.817518,2.547445,3.181818,1


# Rating matrix: 3 Methods

## Cosine

In [20]:
# Subtracting the mean rating of each user from their ratings
RatingMat_centered = RatingMat.sub(RatingMat.mean(axis=1), axis=0)

#user wise similarity with cosine...
user_sim_cos = cosine_similarity(RatingMat)
user_sim_cos_df = pd.DataFrame(user_sim_cos,index=RatingMat.index,columns=RatingMat.index)
user_sim_cos_df.head(5)


Patient_ID,1,2,3,4,5,6,7,8,9,10,...,243,244,245,246,247,248,249,250,251,999
Patient_ID,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.894541,0.941105,0.900155,0.916167,0.910656,0.900076,0.906142,0.906315,0.894628,...,0.868549,0.890518,0.868256,0.857112,0.894977,0.844808,0.878832,0.911458,0.864998,0.318920
2,0.894541,1.000000,0.903770,0.865711,0.854922,0.827976,0.915758,0.902817,0.920410,0.808974,...,0.818879,0.872706,0.799682,0.815612,0.866371,0.791400,0.872935,0.876351,0.843309,0.299303
3,0.941105,0.903770,1.000000,0.937794,0.884078,0.903281,0.916944,0.939743,0.912112,0.860196,...,0.861377,0.878201,0.833596,0.869129,0.932881,0.828436,0.889868,0.909606,0.874682,0.384679
4,0.900155,0.865711,0.937794,1.000000,0.881771,0.902094,0.886630,0.905417,0.872123,0.888419,...,0.871267,0.846654,0.847257,0.871117,0.905814,0.803555,0.896448,0.857597,0.892029,0.273366
5,0.916167,0.854922,0.884078,0.881771,1.000000,0.939121,0.859263,0.875594,0.904124,0.949483,...,0.890029,0.902839,0.865302,0.897344,0.918664,0.869635,0.921692,0.925495,0.903866,0.415101


## Pearson correlation

In [21]:
# Calculate the cosine similarity between each pair of centered ratings
user_sim_pear = 1 - pairwise_distances(RatingMat_centered, metric='correlation')

# Convert the similarity matrix to a Pandas DataFrame with row and column indices corresponding to user IDs
user_sim_pear_df = pd.DataFrame(user_sim_pear, index=RatingMat.index, columns=RatingMat.index)
user_sim_pear_df.head(5)


Patient_ID,1,2,3,4,5,6,7,8,9,10,...,243,244,245,246,247,248,249,250,251,999
Patient_ID,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.417796,0.664229,0.445533,0.589132,0.598594,0.459689,0.536581,0.517297,0.541257,...,0.422291,0.563044,0.413753,0.423496,0.508932,0.323075,0.442934,0.613993,0.385480,0.158631
2,0.417796,1.000000,0.501358,0.320770,0.330339,0.253293,0.582968,0.552085,0.620772,0.195227,...,0.243279,0.505173,0.153397,0.282986,0.410464,0.139401,0.451399,0.477559,0.329921,0.110664
3,0.664229,0.501358,1.000000,0.675980,0.452688,0.577660,0.577250,0.718700,0.570511,0.403470,...,0.413508,0.520857,0.283847,0.492758,0.704036,0.280768,0.516692,0.616067,0.455935,0.315424
4,0.445533,0.320770,0.675980,1.000000,0.452674,0.578553,0.435871,0.562392,0.386839,0.535229,...,0.465418,0.396327,0.355826,0.506899,0.585638,0.187548,0.553520,0.393910,0.540896,0.049530
5,0.589132,0.330339,0.452688,0.452674,1.000000,0.754187,0.359338,0.466075,0.577146,0.803612,...,0.570758,0.642317,0.468535,0.628111,0.664740,0.498003,0.683278,0.704572,0.615637,0.363840


## Euclidean

In [22]:
# user wise similarity with Euclidean distance
user_sim_euc = 1 - pairwise_distances(RatingMat, metric="euclidean")
user_sim_euc_df = pd.DataFrame(user_sim_euc, index=RatingMat.index, columns=RatingMat.index)
user_sim_euc_df.head(5)


Patient_ID,1,2,3,4,5,6,7,8,9,10,...,243,244,245,246,247,248,249,250,251,999
Patient_ID,,,,,,,,,,,,,,,,,,,,,
1,1.000000,-8.201758,-5.960198,-7.978828,-7.263148,-7.947677,-8.008556,-7.714100,-7.720991,-8.254988,...,-9.393595,-8.364552,-9.872257,-9.899794,-8.317430,-10.072968,-8.875700,-7.643802,-9.363149,-18.515384
2,-8.201758,1.000000,-7.297484,-8.412314,-8.874816,-9.279758,-6.401548,-7.252046,-6.245880,-10.050777,...,-9.527836,-8.233253,-9.980459,-9.573837,-9.032847,-10.310964,-7.981355,-7.859032,-8.940838,-16.506644
3,-5.960198,-7.297484,1.000000,-5.696063,-8.112215,-7.401925,-6.695412,-5.653864,-6.918432,-8.866888,...,-8.823307,-8.328452,-9.827527,-8.679231,-6.231259,-9.805061,-7.799248,-7.029355,-8.348250,-16.805354
4,-7.978828,-8.412314,-5.696063,1.000000,-7.864685,-6.792460,-7.513197,-7.115497,-8.122083,-7.391746,...,-7.857142,-9.074476,-8.652314,-7.880626,-7.474203,-9.880822,-7.043755,-8.408909,-7.200435,-16.486344
5,-7.263148,-8.874816,-8.112215,-7.864685,1.000000,-5.452669,-8.593813,-8.349701,-6.982468,-4.771915,...,-7.398138,-7.093208,-8.387278,-7.233763,-6.901173,-8.082910,-6.134923,-5.965663,-6.874943,-15.741021


In [23]:
# 1. select the user we have selected Patient_ID 999 
#Get a list of Feature user already watched so we can remove from our recommendation list..

current_user_rating = dframe[(dframe.Patient_ID == 999) & (dframe.Value != 0)]['Feature']
current_user_rating = pd.DataFrame(current_user_rating, columns=['Feature'])

# Predicting User: 3 Methods

## Cosine

In [36]:
# put similarity of current user i.e. 999 in a dataframe because later we need for weighted average..

curr_user_sim_cos = pd.DataFrame(user_sim_cos_df.loc[999])

#just changing column name as similarity for better readability.
curr_user_sim_cos.rename(columns={999:'Similarity_Score'},inplace=True)
curr_user_sim_cos.sort_values(by='Similarity_Score', ascending=False, inplace=True)
curr_user_sim_cos1 = curr_user_sim_cos.iloc[1:, :]

# Reset index to move it to the first column
curr_user_sim_cos1.reset_index(inplace=True)

# Rename the index column
curr_user_sim_cos1.rename(columns={'index': 'Index_Column'}, inplace=True)

# Define a function to categorize values based on the condition
def categorize(x):
    if x < 165:
        return 'Patient'
    else:
        return 'Healthy'

# Apply the function to the first column of the dataframe
curr_user_sim_cos1['Patient_Type'] = curr_user_sim_cos1['Patient_ID'].apply(categorize)

# Show the resulting dataframe
#curr_user_similarity1.head()

similar_user = curr_user_sim_cos1.iloc[:5, :]
counts_cos = similar_user['Patient_Type'].value_counts()
most_occurring_value = counts_cos.index[0]
print("The user might fall under category of", most_occurring_value)

The user might fall under category of Healthy


## Pearson

In [38]:
# put similarity of current user i.e. 999 in a dataframe because later we need for weighted average..

curr_user_sim_pear = pd.DataFrame(user_sim_pear_df.loc[999])

#just changing column name as similarity for better readability.
curr_user_sim_pear.rename(columns={999:'Similarity_Score'},inplace=True)
curr_user_sim_pear.sort_values(by='Similarity_Score', ascending=False, inplace=True)
curr_user_sim_pear1 = curr_user_sim_pear.iloc[1:, :]

# Reset index to move it to the first column
curr_user_sim_pear1.reset_index(inplace=True)

# Rename the index column
curr_user_sim_pear1.rename(columns={'index': 'Index_Column'}, inplace=True)

# Define a function to categorize values based on the condition
def categorize(x):
    if x < 165:
        return 'Patient'
    else:
        return 'Healthy'

# Apply the function to the first column of the dataframe
curr_user_sim_pear1['Patient_Type'] = curr_user_sim_pear1['Patient_ID'].apply(categorize)

# Show the resulting dataframe
curr_user_sim_pear1.head()

similar_user = curr_user_sim_pear1.iloc[:5, :]
counts_pear = similar_user['Patient_Type'].value_counts()
most_occurring_value = counts_pear.index[0]
print("The user might fall under category of", most_occurring_value)

The user might fall under category of Healthy


## Euclidean

In [44]:
# put similarity of current user i.e. 999 in a dataframe because later we need for weighted average..

curr_user_sim_euc = pd.DataFrame(user_sim_euc_df.loc[999])

#just changing column name as similarity for better readability.
curr_user_sim_euc.rename(columns={999:'Similarity_Score'},inplace=True)
curr_user_sim_euc.sort_values(by='Similarity_Score', ascending=False, inplace=True)
curr_user_sim_euc1 = curr_user_sim_euc.iloc[1:, :]

# Reset index to move it to the first column
curr_user_sim_euc1.reset_index(inplace=True)

# Rename the index column
curr_user_sim_euc1.rename(columns={'index': 'Index_Column'}, inplace=True)

# Define a function to categorize values based on the condition
def categorize(x):
    if x < 165:
        return 'Patient'
    else:
        return 'Healthy'

# Apply the function to the first column of the dataframe
curr_user_sim_euc1['Patient_Type'] = curr_user_sim_euc1['Patient_ID'].apply(categorize)

# Show the resulting dataframe
curr_user_sim_euc1.head()

similar_user = curr_user_sim_euc1.iloc[:5, :]
counts_euc = similar_user['Patient_Type'].value_counts()
most_occurring_value = counts_euc.index[0]

print("The user might fall under category of", most_occurring_value)

The user might fall under category of Healthy


In [52]:
def categorize(x):
    if x < 165:
        return 'Patient'
    else:
        return 'Healthy'

# put similarity of current user i.e. 999 in a dataframe because later we need for weighted average..
curr_user_sim_cos = pd.DataFrame(user_sim_cos_df.loc[999])
curr_user_sim_cos.rename(columns={999:'Similarity_Score'}, inplace=True)
curr_user_sim_cos.sort_values(by='Similarity_Score', ascending=False, inplace=True)
curr_user_sim_cos1 = curr_user_sim_cos.iloc[1:, :]
curr_user_sim_cos1.reset_index(inplace=True)
curr_user_sim_cos1.rename(columns={'index': 'Index_Column'}, inplace=True)
curr_user_sim_cos1['Patient_Type'] = curr_user_sim_cos1['Patient_ID'].apply(categorize)

curr_user_sim_pear = pd.DataFrame(user_sim_pear_df.loc[999])
curr_user_sim_pear.rename(columns={999:'Similarity_Score'}, inplace=True)
curr_user_sim_pear.sort_values(by='Similarity_Score', ascending=False, inplace=True)
curr_user_sim_pear1 = curr_user_sim_pear.iloc[1:, :]
curr_user_sim_pear1.reset_index(inplace=True)
curr_user_sim_pear1.rename(columns={'index': 'Index_Column'}, inplace=True)
curr_user_sim_pear1['Patient_Type'] = curr_user_sim_pear1['Patient_ID'].apply(categorize)

curr_user_sim_euc = pd.DataFrame(user_sim_euc_df.loc[999])
curr_user_sim_euc.rename(columns={999:'Similarity_Score'}, inplace=True)
curr_user_sim_euc.sort_values(by='Similarity_Score', ascending=False, inplace=True)
curr_user_sim_euc1 = curr_user_sim_euc.iloc[1:, :]
curr_user_sim_euc1.reset_index(inplace=True)
curr_user_sim_euc1.rename(columns={'index': 'Index_Column'}, inplace=True)
curr_user_sim_euc1['Patient_Type'] = curr_user_sim_euc1['Patient_ID'].apply(categorize)

# Get the most occurring value based on the vote of the three outputs
similar_user_cos = curr_user_sim_cos1.iloc[:5, :]
similar_user_pear = curr_user_sim_pear1.iloc[:5, :]
similar_user_euc = curr_user_sim_euc1.iloc[:5, :]

counts_cos = similar_user_cos['Patient_Type'].value_counts()
counts_pear = similar_user_pear['Patient_Type'].value_counts()
counts_euc = similar_user_euc['Patient_Type'].value_counts()

counts_all = pd.concat([counts_cos, counts_pear, counts_euc], axis=1).fillna(0)
total_counts = counts_all.sum(axis=1)
most_occurring_value = total_counts.idxmax()

print("The user might fall under category of", most_occurring_value)
print(total_counts)


The user might fall under category of Healthy
Healthy    11
Patient     4
dtype: int64


In [56]:
total_counts = pd.DataFrame(total_counts)
total_counts.head()

,0
Healthy,11
Patient,4


# Predicting Scoring of user: Top users of its category

In [ ]:
select_simil_user = similar_user.loc[similar_user['Patient_Type'] == most_occurring_value]

In [ ]:
df_simil = df1[df1['Patient_ID'].isin(select_simil_user['Patient_ID'])]
df_simil.head()

In [ ]:
similar_user.head()

In [ ]:
similar_user = similar_user.drop(['Patient_Type'], axis=1)
similar_user.head(2)

In [ ]:
df_score = pd.merge(similar_user, df_simil, on = 'Patient_ID', how = 'inner')
df_score.head()

In [ ]:
# Example DataFrame Y with some filled and some NaN values
X = df_simil
Y = df3.drop(['Patient_ID', 'Age'], axis = 1)

# Extract the weights from the second column of df_score i.e. extracting Similarity_Score
weights = df_score.iloc[:, 1]

# Extract the values to be averaged from the remaining columns of X
values = X.iloc[:, 2:]

# Compute the weighted average of each column
weighted_avg_cols = np.average(values, axis=0, weights=weights)

# Create a dictionary that maps column names to weighted average values
fill_values = {col: val for col, val in zip(Y.columns, weighted_avg_cols)}

# Fill NaN values in DataFrame Y with the mean values
Y = Y.fillna(fill_values)
col_Age = df3.pop('Age')
Y.insert(0, 'Age', col_Age)
Y

colX = df_simil
colY = df3.drop(['Patient_ID', 'Age'], axis = 1)
col_weights = df_score.iloc[:, 1]
values = colX.iloc[:, 2:]
weighted_avg_cols = np.average(values, axis=0, weights=col_weights)
fill_values = {col: val for col, val in zip(colY.columns, weighted_avg_cols)}
colY = colY.fillna(fill_values)
col_Age = df3.pop('Age')
colY.insert(0, 'Age', col_Age)
colY